In [1]:
import numpy_financial as npf

In [2]:
def monthly_PI(interest, years, price, yearly_months=12):
    monthly_PI = -1 * npf.pmt(interest/yearly_months, years*yearly_months, price)
    return monthly_PI

In [3]:
# purchase
purchase_price = 600e3
downpayment = 21e3
rehab_cost = 100e3
after_repair_value = 800e3

# acquisition
yearly_months = 12
acq_yearly_interest = 0 # 0.065
mortgage_years = 30
rehab_months = 6
monthly_taxes = 555 # from zillow

# # initial rental period
monthly_rent = 4e3
vacancy_frac, repairs_frac, capex_frac = 0.05, 0.05, 0.05

# refinanced rental period 
refinance_months = 9
pre_refi_duration = refinance_months-rehab_months

In [4]:
class Mortgage():
    def __init__(rate, years, price):
        self.org_monthly_PI = self.monthly_PI(rate, years, price)
    def original_contract(self, rate, years, price, yearly_months=yearly_months):
        monthly_PI = -1 * npf.pmt(rate/yearly_months, years*yearly_months, price)
        return monthly_PI

In [5]:
class Acquisition():
    """ Could inherent a Mortgage object if it could contain several methods """
    def __init__(self, purchase_price, downpayment, yearly_interest, monthly_taxes):
        self.time = {}
        self.price = {
            'home_value': purchase_price,
            'downpayment': downpayment,
            'mortgage': np.nan,
            'monthly_PI': np.nan,
            'monthly_taxes': monthly_taxes,
            
        } 
        self.exponent ={
            'yearly_interest': yearly_interest,
        }
        
        self.derive_properties()
        
    def derive_properties(self):
#         super().__init__(self.price['yearly_interest'], 30, self.price['price'])
        self.price['loan_fees'] = 0.01 * self.price['home_value']
        self.price['mortgage'] = self.price['home_value'] - self.price['downpayment'] + self.price['loan_fees']
        self.price['monthly_PI'] = monthly_PI(self.exponent['yearly_interest'], 30, self.price['mortgage'])
        self.price['closing'] = self.price['home_value'] * 0.01 
        
class Rehab():
    def __init__(self, rehab_months, total_cost, monthly_PI, monthly_taxes, monthly_insurance=0, other_costs = 0):
        self.time = {
            'total_months' : rehab_months
        }
        self.price= {
            'total_cost': total_cost,
            'monthly_insurance': monthly_insurance,
            'other': other_costs,
            'holding_cost': np.nan,
            'monthly_PI': monthly_PI,
            'monthly_taxes': monthly_taxes
        } 
        self.exponent = {
        }

        self.derive_properties()
        
    def derive_properties(self):
        self.price['monthly_rehab'] = self.price['total_cost']*self.time['total_months']/yearly_months
        self.price['holding_cost'] = (self.price['monthly_PI'] + self.price['monthly_taxes']) * self.time['total_months']
        self.price['monthly_total'] = self.price['monthly_rehab'] + self.price['monthly_taxes'] + self.price['monthly_PI']
        
class PreReFi_Rent():
    """ Could inherent a Mortgage object if """
    def __init__(self, monthly_rent, vacancy_frac, repairs_frac, capex_frac, total_time, monthly_taxes, 
                monthly_PI):
        self.time = {
            'total': total_time
        }
        self.price =  {
            'monthly_rent': monthly_rent,
            'vacancy_frac': vacancy_frac,
            'repairs_frac': repairs_frac,
            'capex_frac': capex_frac,
            'monthly_taxes': monthly_taxes,
            'monthly_PI': monthly_PI
        } 
        self.exponent = {}
        self.derive_properties()
        
    def derive_properties(self):
        self.price['monthly_vacancy'] = self.price['monthly_rent']*self.price['vacancy_frac']
        self.price['monthly_repairs'] = self.price['monthly_rent']*self.price['repairs_frac']
        self.price['monthly_capex'] = self.price['monthly_rent']*self.price['capex_frac']
        self.price['monthly_expenses'] = self.sum_expenses()
        self.price['monthly_cashflow'] = self.price['monthly_rent'] - self.price['monthly_expenses']
        self.price['NOI'] = (self.price['monthly_cashflow'] + self.price['monthly_PI']) * yearly_months  # googled NOI vs cash flow

    def sum_expenses(self):
        exp_sum = self.price['monthly_vacancy'] + self.price['monthly_capex'] \
        + self.price['monthly_taxes'] + self.price['monthly_repairs'] + self.price['monthly_PI']
        return exp_sum

class Refinance():
    """ Could inherent a Mortgage object if """
    def __init__(self, monthly_rent, home_value, vacancy_frac, repairs_frac, capex_frac, monthly_taxes):
        self.time =  {}
        self.price =  {
            'monthly_rent': monthly_rent,
            'home_value': home_value,
            'vacancy_frac': vacancy_frac,
            'repairs_frac': repairs_frac,
            'capex_frac': capex_frac,
            'monthly_PI': np.nan,
            'monthly_taxes': monthly_taxes
        } 
        self.exponent = {
            'yearly_interest': 0#0.065
        }
        self.derive_properties()

    def derive_properties(self):
        self.price['monthly_vacancy'] = self.price['monthly_rent']*self.price['vacancy_frac']
        self.price['monthly_repairs'] = self.price['monthly_rent']*self.price['repairs_frac']
        self.price['monthly_capex'] = self.price['monthly_rent']*self.price['capex_frac']
        self.price['loan_fees'] = 0.01 * self.price['home_value']
        self.price['mortgage'] = 0.8 * self.price['home_value'] + self.price['loan_fees']
        self.price['monthly_PI'] = monthly_PI(self.exponent['yearly_interest'], 30, self.price['mortgage'])
        self.price['monthly_expenses'] = self.sum_expenses()
        self.price['monthly_cashflow'] = self.price['monthly_rent'] - self.price['monthly_expenses']
        self.price['NOI'] = (self.price['monthly_cashflow'] + self.price['monthly_PI']) * yearly_months  # googled NOI vs cash flow

    def sum_expenses(self):
        exp_sum = self.price['monthly_vacancy'] + self.price['monthly_capex'] \
        + self.price['monthly_taxes'] + self.price['monthly_repairs'] + self.price['monthly_PI']
        return exp_sum

In [6]:
acq = Acquisition(purchase_price, downpayment, acq_yearly_interest, monthly_taxes)
rehab = Rehab(rehab_months, rehab_cost, acq.price['monthly_PI'], monthly_taxes)
pre_refi = PreReFi_Rent(monthly_rent, vacancy_frac, repairs_frac, capex_frac, 
                        pre_refi_duration, monthly_taxes, acq.price['monthly_PI'])
refi = Refinance(monthly_rent, after_repair_value, vacancy_frac, repairs_frac, capex_frac, monthly_taxes)

In [7]:
rehab.price['total_cost'],  acq.price['downpayment']

(100000.0, 21000.0)

In [8]:
cash_required = acq.price['downpayment'] + rehab.price['total_cost'] + acq.price['closing']
cash_outlay = cash_required + rehab.price['holding_cost']
acq_cash_on_cash_roi = pre_refi.price['monthly_cashflow']*yearly_months/cash_required
ref_cash_on_cash_roi = refi.price['monthly_cashflow']*yearly_months/cash_required

pro_forma_cap_rate = pre_refi.price['NOI']/refi.price['mortgage']
purchase_cap_rate = pre_refi.price['NOI']/acq.price['mortgage']

In [9]:
pre_refi.price['NOI'], pro_forma_cap_rate, purchase_cap_rate, acq.price['mortgage'], refi.price['mortgage']

(34140.0, 0.05268518518518518, 0.05835897435897436, 585000.0, 648000.0)

In [10]:
refi.__dict__

{'time': {},
 'price': {'monthly_rent': 4000.0,
  'home_value': 800000.0,
  'vacancy_frac': 0.05,
  'repairs_frac': 0.05,
  'capex_frac': 0.05,
  'monthly_PI': 1800.0,
  'monthly_taxes': 555,
  'monthly_vacancy': 200.0,
  'monthly_repairs': 200.0,
  'monthly_capex': 200.0,
  'loan_fees': 8000.0,
  'mortgage': 648000.0,
  'monthly_expenses': 2955.0,
  'monthly_cashflow': 1045.0,
  'NOI': 34140.0},
 'exponent': {'yearly_interest': 0}}

In [11]:
acq.__dict__, pre_refi.__dict__

({'time': {},
  'price': {'home_value': 600000.0,
   'downpayment': 21000.0,
   'mortgage': 585000.0,
   'monthly_PI': 1625.0,
   'monthly_taxes': 555,
   'loan_fees': 6000.0,
   'closing': 6000.0},
  'exponent': {'yearly_interest': 0}},
 {'time': {'total': 3},
  'price': {'monthly_rent': 4000.0,
   'vacancy_frac': 0.05,
   'repairs_frac': 0.05,
   'capex_frac': 0.05,
   'monthly_taxes': 555,
   'monthly_PI': 1625.0,
   'monthly_vacancy': 200.0,
   'monthly_repairs': 200.0,
   'monthly_capex': 200.0,
   'monthly_expenses': 2780.0,
   'monthly_cashflow': 1220.0,
   'NOI': 34140.0},
  'exponent': {}})

In [12]:
print('Summary')
print(f"Purchase Closing Costs: ${acq.price['closing']}")
print(f"Estimated Repairs: ${rehab.price['total_cost']}")
print(f"After Repair Value: ${refi.price['home_value']}")

Summary
Purchase Closing Costs: $6000.0
Estimated Repairs: $100000.0
After Repair Value: $800000.0


In [13]:
print('Acquisition')
print(f"Down Payment: ${acq.price['downpayment']}")
print(f"Loan Amount: ${acq.price['mortgage']}")
print(f"Loan Points/Fees: ${acq.price['loan_fees']}")
print(f"Loan Interest Rate: {acq.exponent['yearly_interest']*100}%")
print(f"Monthly P&I: ${acq.price['monthly_PI']:.2f}")
print(f"Total Cash needed at purchase: ${cash_required}")

Acquisition
Down Payment: $21000.0
Loan Amount: $585000.0
Loan Points/Fees: $6000.0
Loan Interest Rate: 0%
Monthly P&I: $1625.00
Total Cash needed at purchase: $127000.0


In [14]:
print('Rehab Period')
print(f"Holding costs: ${rehab.price['holding_cost']:.2f}")
print(f"Total cash outlay: ${cash_outlay}")
print(f"Monthly P&I: ${acq.price['monthly_PI']:.2f}")
print(f"Rehab time: {rehab.time['total_months']} months")
print(f"Monthly taxes: ${monthly_taxes}")

Rehab Period
Holding costs: $13080.00
Total cash outlay: $140080.0
Monthly P&I: $1625.00
Rehab time: 6 months
Monthly taxes: $555


In [15]:
refi.__dict__

{'time': {},
 'price': {'monthly_rent': 4000.0,
  'home_value': 800000.0,
  'vacancy_frac': 0.05,
  'repairs_frac': 0.05,
  'capex_frac': 0.05,
  'monthly_PI': 1800.0,
  'monthly_taxes': 555,
  'monthly_vacancy': 200.0,
  'monthly_repairs': 200.0,
  'monthly_capex': 200.0,
  'loan_fees': 8000.0,
  'mortgage': 648000.0,
  'monthly_expenses': 2955.0,
  'monthly_cashflow': 1045.0,
  'NOI': 34140.0},
 'exponent': {'yearly_interest': 0}}

In [16]:
print('Refinance')
print(f"Loan Amount: ${refi.price['mortgage']}")
print(f"Loan Points/Fees: ${refi.price['loan_fees']}")
print(f"Loan Interest Rate: {refi.exponent['yearly_interest']*100}%")
print(f"Monthly P&I: ${refi.price['monthly_PI']:.2f}")

Refinance
Loan Amount: $648000.0
Loan Points/Fees: $8000.0
Loan Interest Rate: 0%
Monthly P&I: $1800.00


In [17]:
acq.__dict__, rehab.__dict__, pre_refi.__dict__

({'time': {},
  'price': {'home_value': 600000.0,
   'downpayment': 21000.0,
   'mortgage': 585000.0,
   'monthly_PI': 1625.0,
   'monthly_taxes': 555,
   'loan_fees': 6000.0,
   'closing': 6000.0},
  'exponent': {'yearly_interest': 0}},
 {'time': {'total_months': 6},
  'price': {'total_cost': 100000.0,
   'monthly_insurance': 0,
   'other': 0,
   'holding_cost': 13080.0,
   'monthly_PI': 1625.0,
   'monthly_taxes': 555,
   'monthly_rehab': 50000.0,
   'monthly_total': 52180.0},
  'exponent': {}},
 {'time': {'total': 3},
  'price': {'monthly_rent': 4000.0,
   'vacancy_frac': 0.05,
   'repairs_frac': 0.05,
   'capex_frac': 0.05,
   'monthly_taxes': 555,
   'monthly_PI': 1625.0,
   'monthly_vacancy': 200.0,
   'monthly_repairs': 200.0,
   'monthly_capex': 200.0,
   'monthly_expenses': 2780.0,
   'monthly_cashflow': 1220.0,
   'NOI': 34140.0},
  'exponent': {}})

In [18]:
print('Initial Rental Period')
print(f"Monthly Income: ${pre_refi.price['monthly_rent']}")
print(f"Monthly Expenses: ${pre_refi.price['monthly_expenses']:.2f}")
print(f"Monthly Cashflow: ${pre_refi.price['monthly_cashflow']:.2f}")
print(f"NOI: ${pre_refi.price['NOI']}")
print(f"Cash on cash ROI: {acq_cash_on_cash_roi*100:.3g}%")
print(f"Pro Forma Cap Rate: {pro_forma_cap_rate*100:.3g}%")
print(f"Purchase Cap Rate: {purchase_cap_rate*100:.3g}%")

Initial Rental Period
Monthly Income: $4000.0
Monthly Expenses: $2780.00
Monthly Cashflow: $1220.00
NOI: $34140.0
Cash on cash ROI: 11.5%
Pro Forma Cap Rate: 5.27%
Purchase Cap Rate: 5.84%


In [19]:
refi.__dict__

{'time': {},
 'price': {'monthly_rent': 4000.0,
  'home_value': 800000.0,
  'vacancy_frac': 0.05,
  'repairs_frac': 0.05,
  'capex_frac': 0.05,
  'monthly_PI': 1800.0,
  'monthly_taxes': 555,
  'monthly_vacancy': 200.0,
  'monthly_repairs': 200.0,
  'monthly_capex': 200.0,
  'loan_fees': 8000.0,
  'mortgage': 648000.0,
  'monthly_expenses': 2955.0,
  'monthly_cashflow': 1045.0,
  'NOI': 34140.0},
 'exponent': {'yearly_interest': 0}}

In [20]:
print('Refinance Rental Period')
print(f"Monthly Income: ${refi.price['monthly_rent']}")
print(f"Monthly Expenses: ${refi.price['monthly_expenses']:.2f}")
print(f"Monthly Cashflow: ${refi.price['monthly_cashflow']:.2f}")
print(f"NOI: ${refi.price['NOI']}")
print(f"Cash on cash ROI: {ref_cash_on_cash_roi*100:.3g}%")
print(f'Pro Forma Cap Rate: {pro_forma_cap_rate*100:.3g}%')
print(f'Purchase Cap Rate: {purchase_cap_rate*100:.3g}%')

Refinance Rental Period
Monthly Income: $4000.0
Monthly Expenses: $2955.00
Monthly Cashflow: $1045.00
NOI: $34140.0
Cash on cash ROI: 9.87%
Pro Forma Cap Rate: 5.27%
Purchase Cap Rate: 5.84%
